In [2]:
from __future__ import absolute_import 
from __future__ import division
from __future__ import print_function
import tensorflow as tf
from scipy import misc
import cv2
import numpy as np 
import facenet 
import detect_face
import os
import time 
import pickle

In [3]:
input_video="test_video_1.mp4"
modeldir='./model/20170511-185253.pb'
classifier_filename='./vclass/classifier.pkl'
npy='./npy'
train_img="./train_img"

In [6]:
with tf.Graph().as_default():
    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess=tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options,log_device_placement=False))
    with sess.as_default():
        pnet,rnet,onet=detect_face.create_mtcnn(sess,npy)
        minsize=20
        threshold=[0.6,0.7,0.7]
        factor=0.709
        margin=44
        frame_interval=3
        batch_size=182
        input_image_size=160
        HumanNames=os.listdir(train_img)
        HumanNames.sort()
        print('Loading Model')
        facenet.load_model(modeldir)
        image_placeholder=tf.compat.v1.get_default_graph().get_tensor_by_name("input:0")
        emdeddings=tf.compat.v1.get_default_graph().get_tensor_by_name("emdeddings:0")
        phase_train_placeholder=tf.compat.v1.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size=embeddings.get_shape()[1]
        
        classifer_filename_exp=os.path.expanduser(classifier_filename)
        with open (classifier_filename_exp,'rb') as infile:
            (model,class_names)=pickle.load(infile)
        video_capture=cv2.VideoCapture(input_video)
        c=0
        print('Start Recognition')
        prevTime=0
        while True :
            ret,frame=video_capture(input_video)
            frame=cv2.resize(frame,(0,0),fx=0.5,fy=0.5)
            curTime=time.time()+1
            timeF=frame_interval
            if(c% timeF==0):
                find_results=[]
                if feame.ndim==2:
                    frame=facenet.to_rgb(frame)
                frame=frame[:,:,0:3]
                bounding_boxes=detect_face.detect_face(frame,minsize,pnet,rnet,onet,threshold,factor)
                nrof_faces=bounding_boxes.shape[0]
                print('Detect_FaceNum: %d' %nrof_faces)
                if nrof_faces > 0:
                    det=boinding_boxes[:,0:4]
                    img_size=np.asarray(frame.shape)[0:2]
                    cropped=[]
                    scaled=[]
                    scaled_reshape=[]
                    bb=np.zeros((nrof_faces,4),dtype=np.int32)
                    
                    for i in range (nrof_faces):
                        emb_array=np.zeros((1,embedding_size))
                        
                        b[i][0]=det[i][0]
                        b[i][1]=det[i][1]
                        b[i][2]=det[i][2]
                        b[i][3]=det[i][3]
                        
                        if bb[i][0]<=0 or bb[i][1]<=0 or bb[i][2] >= len(frame[0]) or bb[i][3] >= len(frame):
                            print('Face is very close !')
                            continue
                        cropped.append(frame[bb[i][1] :bb[i][3] ,bb[i][0]:b[i][2] ,:])
                        cropped[i]=facenet.flip(cropped[i],False)
                        scaled.append(misc.imresize(cropped[i],(image_size,image_size),interp='bilinear'))
                        scaled[i]=cv2.resize(scaled[i],(input_image_size,input_image_size),interpolation=cv2.INTER_CUBIC)
                        scaled[i]=facenet.prewhiten(scaled[i])
                        scaled_reshape.append(scaled[i].reshape(-1,input_image_size,input_image_size,3))
                        feed_dict={images_placeholder:scaled_reshape[i] , phase_train_placeholder:False}
                        emb_array[0,:]=sess.run(embeddings,feed_dict=feed_dict)
                        predictions=model.predict_proba(emb_array)
                        print(predictions)
                        best_class_indices=np.argmax(predictions,axis=1)
                        best_class_probabilies=predictions[np.arange(len(best_class_indices)),best_class_indices]
                        print(nest_class_indices,'with accuracy',best_class_probabilities)
                        if best_class_probabilities >0.53:
                            cv2.rectangle(frame,(bb[i][0],bb[i][1]),(bb[i][2],bb[i][3]),(0,255,0),2)
                            
                            text_x=bb[i][0]
                            text_y=bb[i][3] +20
                            print('Result Indices: ',best_class_indices[0])
                            print(HumanNames)
                            for H_i in HumanNames:
                                if HumanNames[best_class_indices[0]]==H_i:
                                    result_names=HumanNames[best_class_indices[0]]
                                    cv2.putText(frame,result_names,(text_x,text_y) , 
                                               cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                               1,(0,0,255) thickness=1,lineType=2)
                else :
                    print('Alignment Failure')
                    
             cv2.imshow('Video',frame)
            
             if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        video_capture.release()
        cv2.destroyAllWindows()
           
                            
                            
                            